In [ ]:
# @title Init environment { display-mode: "form" }
!python -m pip install --upgrade pip setuptools wheel && python -m pip install lbry-libtorrent wget torf && apt install python3-libtor
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Set download path { display-mode: "form" }
file_path = '/proj/anime-dl' #@param {type:"string"}
root_path = '/content/drive/MyDrive'+file_path
magnets = []

In [ ]:
# @title Clear Download List { display-mode: "form" }
magnets = []
print(magnets)

In [ ]:
# @title Append Magnet Link { display-mode: "form" }
magnet = '' #@param {type:"string"}
if magnet not in magnets:
    magnets.append(magnet)
else:
    print("Already Exist!")
print(f"Current magnet list count: {len(magnets)}")
for mag in magnets:
    print(f"{mag}")

In [ ]:
# @title Run Download { display-mode: "form" }
import time
from IPython.display import display
import ipywidgets as widgets
import libtorrent as lt

ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []
params = {"save_path": root_path}

for mag_link in magnets:
    downloads.append(
        lt.add_magnet_uri(ses, mag_link, params)
    )

state_str = [
    "queued",
    "checking",
    "downloading metadata",
    "downloading",
    "finished",
    "seeding",
    "allocating",
    "checking fastresume",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    download.name(),
                    str(s.download_rate / 1000),
                    "kB/s",
                    state_str[s.state],
                ]
            )
            bar.value = s.progress * 100
        else:
            next_shift -= 1
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() # Seems to be not working in Colab (see https://github.com/googlecolab/colabtools/issues/726#issue-486731758)
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)
magnets = []